In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers==4.28.0;
!pip install sentencepiece;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.5 MB/s eta 0:00:00


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import json
from copy import deepcopy
import random
from tqdm.notebook import tqdm

from transformers import (
    PreTrainedTokenizerFast as BaseGPT2Tokenizer,
    EncoderDecoderModel,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    AdamW
)
import pickle

from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai.tokenization_kobert import KoBertTokenizer
from drive.MyDrive.WARNING_PRIVATE_FOLDER.OSSP2AntiGPT9.ai import tokenization_kobert
src_tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [5]:

class GPT2Tokenizer(BaseGPT2Tokenizer):
    def build_inputs_with_special_tokens(self, token_ids, _):
        return token_ids + [self.eos_token_id]
trg_tokenizer = GPT2Tokenizer.from_pretrained("skt/kogpt2-base-v2",
  bos_token='</s>', eos_token='</s>', unk_token='<unk>',
  pad_token='<pad>', mask_token='<mask>')

In [6]:
with open("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/standard_to_dialect/no_commit/added_tokens_trg.json", 'rb') as f:
    dw_dict = json.load(f)
dict_swap = {v:k for k,v in dw_dict.items()}
ma=54000
mi=51200
for i in range(51200,54001):
    trg_tokenizer.add_tokens(dict_swap[i])

In [7]:
%cd /content
!pwd

/content
/content


## 데이터 불러오기 && 데이터셋 만들기 

In [8]:
def split_input_dict(input_dict, ratio = 0.01, seed = 42):
    split_point = int(len(input_dict['utterance']) * ratio)
    random.seed(seed)
    random.shuffle(input_dict['utterance'])
    valid_dict = deepcopy(input_dict)
    train_dict = input_dict

    valid_dict['utterance'] = input_dict['utterance'][:split_point]
    train_dict['utterance'] = input_dict['utterance'][split_point:]
    return train_dict, valid_dict

In [9]:
def read_input(path):
    with open(path, 'rb') as f:
        input_dict = json.load(f)
    train_dict,valid_dict = split_input_dict(input_dict)
    train_standard = []
    train_dialect = []
    for pairs in tqdm(train_dict['utterance']):
        train_standard.append(pairs['standard_form'])
        train_dialect.append(pairs['dialect_form'])

    valid_standard = []
    valid_dialect = []
    for pairs in tqdm(valid_dict['utterance']):
        valid_standard.append(pairs['standard_form'])
        valid_dialect.append(pairs['dialect_form'])

    
    return train_standard ,train_dialect , valid_standard , valid_dialect

In [10]:
train_s ,train_d , valid_s , valid_d = read_input("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/real_data.json")

  0%|          | 0/1220349 [00:00<?, ?it/s]

  0%|          | 0/12326 [00:00<?, ?it/s]

In [11]:
class PairedDataset:
    def __init__(self, src_tokenizer, tgt_tokenizer,file_standard , file_dialect):
        self.src_tokenizer = src_tokenizer
        self.trg_tokenizer = tgt_tokenizer
        self.data_s = file_standard
        self.data_d = file_dialect
        self.data = [[self.data_s[i],self.data_d[i]] for i in range(len(self.data_s)) ]
    def __getitem__(self, index):
        src, trg = self.data_s[index] , self.data_d[index]
        embeddings = self.src_tokenizer(src, return_attention_mask=False, return_token_type_ids=False)
        embeddings['labels'] = self.trg_tokenizer(trg, return_attention_mask=False)['input_ids']

        return embeddings

    def __len__(self):
        return len(self.data)

In [12]:
train_s ,train_d , valid_s , valid_d = read_input("/content/drive/MyDrive/WARNING_PRIVATE_FOLDER/OSSP2AntiGPT9/ai/no_commit/real_data.json")
dataset = PairedDataset(src_tokenizer, trg_tokenizer, train_s , train_d)
eval_dataset = PairedDataset(src_tokenizer, trg_tokenizer, valid_s , valid_d)

  0%|          | 0/1220349 [00:00<?, ?it/s]

  0%|          | 0/12326 [00:00<?, ?it/s]

## 사전학습 모델 불러오기

In [13]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained(
    'monologg/kobert',
    'skt/kogpt2-base-v2',
    pad_token_id=trg_tokenizer.bos_token_id
)
model.config.decoder_start_token_id = trg_tokenizer.bos_token_id
model.encoder.resize_token_embeddings(len(src_tokenizer))
model.decoder.resize_token_embeddings(len(trg_tokenizer))
# # 토크나이저들을 모델에 저장합니다
model.config.encoder_attention_type = 'full'
model.config.decoder_attention_type = 'full'
model.config.src_vocab_size = len(src_tokenizer)
model.config.tgt_vocab_size = len(trg_tokenizer)

# push_to_hub() 함수를 사용하여 모델을 Hugging Face Hub에 업로드합니다
#model.push_to_hub("ossp-reverse-v0_2")

Some weights of GPT2LMHeadModel were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['transformer.h.2.crossattention.c_attn.weight', 'transformer.h.1.ln_cross_attn.weight', 'transformer.h.2.crossattention.c_proj.weight', 'transformer.h.3.crossattention.c_proj.bias', 'transformer.h.8.ln_cross_attn.weight', 'transformer.h.5.ln_cross_attn.weight', 'transformer.h.7.crossattention.c_proj.weight', 'transformer.h.6.crossattention.c_proj.weight', 'transformer.h.9.crossattention.c_proj.bias', 'transformer.h.9.crossattention.c_proj.weight', 'transformer.h.11.crossattention.q_attn.weight', 'transformer.h.3.crossattention.masked_bias', 'transformer.h.10.ln_cross_attn.weight', 'transformer.h.7.crossattention.q_attn.weight', 'transformer.h.4.ln_cross_attn.weight', 'transformer.h.4.crossattention.c_proj.bias', 'transformer.h.10.crossattention.c_proj.bias', 'transformer.h.10.crossattention.bias', 'transformer.h.3.crossattention.bias', 'transformer.h.9.ln_cro

## 학습 코드

In [14]:
!pwd
%cd /content

/content
/content


In [15]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 13.0 MB/s eta 0:00:00


In [17]:
collator = DataCollatorForSeq2Seq(src_tokenizer, model)

arguments = Seq2SeqTrainingArguments(
    output_dir='ossp-reverse-v0_3',
    do_train=True,
    do_eval=True,
    evaluation_strategy="steps",
    eval_steps=10000,
    save_strategy="steps",
    save_steps=10000,
    num_train_epochs=2,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    warmup_ratio=0.1,
    gradient_accumulation_steps=1,
    save_total_limit=1,
    dataloader_num_workers=1,
    fp16=True,
    load_best_model_at_end=True,
    push_to_hub=True,
    learning_rate = 5e-5
)

trainer = Seq2SeqTrainer(
    model,
    arguments,
    data_collator=collator,
    train_dataset=dataset,
    eval_dataset=eval_dataset
)

/content/ossp-reverse-v0_3 is already a clone of https://huggingface.co/leadawon/ossp-reverse-v0_3. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

model.save_pretrained("ossp-reverse-v0_3/best_model")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Step,Training Loss,Validation Loss
10000,1.226600,1.094519
20000,0.848500,0.783250
30000,0.754200,0.710081
40000,0.686300,0.662318
50000,0.668900,0.627242
60000,0.601700,0.607513
70000,0.585800,0.589496


/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:634: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no 

In [ ]:
trainer.push_to_hub()